In [3]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
import pickle
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential, Model
from tensorflow.keras.constraints import max_norm

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
df_short = pd.read_csv("data/processed/processed_short.csv")
df_medium = pd.read_csv("data/processed/processed_medium.csv")
df_dank = pd.read_csv("data/processed/processed_dank.csv")


In [26]:
def build_model():
    init_matrix = pickle.load(open( "init_matrix.p", "rb" ))
    submodels = []
    for kw in (3, 4, 5):    # kernel sizes
        submodel = Sequential()
        submodel.add(Embedding(
        30002,
        100,
        embeddings_initializer=keras.initializers.Constant(init_matrix),
        trainable=False
    ))
        submodel.add(Conv1D(100,    #should be 100 maps
                            kw,
                            padding='valid',
                            activation='relu',
                            strides=1, kernel_constraint=max_norm(3.7))) #elvileg egy maxnorm nem árt még bele
        submodel.add(GlobalMaxPooling1D())
        submodels.append(submodel)

    submodel1 = submodels[0]
    submodel2 = submodels[1]
    submodel3 = submodels[2]

    x = add([submodel1.output, submodel2.output, submodel3.output])
    
    big_model = Sequential()
    big_model.add(Dropout(0.25))
    big_model.add(Dense(50))
    big_model.add(Dropout(0.6))
    big_model.add(Activation('relu'))
    big_model.add(Dense(1))
    big_model.add(Activation('sigmoid'))

    big_model_output = big_model(x)

    model = Model([submodel1.input, submodel2.input, submodel3.input], big_model_output)

    model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])  

    print(model.summary())  


    return model

In [11]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split

vectorizer = TextVectorization(max_tokens=30000, output_sequence_length=200)

In [12]:
def train_model(df):
    name =[x for x in globals() if globals()[x] is df][0]
    #creating the desired vectors
    text = np.asarray(df['text'])
    y = df["label"]
    text_train, text_test, y_train, y_test = train_test_split(
    text, y, test_size=0.33, random_state=42)

    text_train, text_val, y_train, y_val = train_test_split(
    text_train, y_train, test_size=0.33, random_state=42)

    X_train = vectorizer(text_train)
    X_val = vectorizer(text_val)
    X_test = vectorizer(text_test)

    model = build_model()

    model.fit([X_train, X_train, X_train],
                     y_train,
                     batch_size=64,
                     epochs=25,
                     validation_data=([X_val, X_val, X_val],
                     y_val))
    
    loss, accuracy =  model.evaluate([X_test, X_test, X_test], y_test)

    print("Loss: ", loss)
    print("Accuracy: ", accuracy)
        
    return model



In [25]:
train_model(df_medium)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_10_input (InputLayer) [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_11_input (InputLayer) [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_12_input (InputLayer) [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, None, 100)    3000200     embedding_10_input[0][0]         
____________________________________________________________________________________________